# Ancient to Modern Italian Translation with Mistral

This notebook implements Mistral for translating ancient Italian into modern Italian:

**Mistral (Fine-tuned and optimized with LoRa)**

---


# Imports

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig
import pandas as pd
from datasets import load_dataset
from sklearn.metrics import cohen_kappa_score
import torch
from tqdm import tqdm
from peft import get_peft_model, LoraConfig, TaskType
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling, logging
import glob
import os


c:\Users\colam\Documents\GitHub\MNLP_HW2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load dataset

In [2]:
dataset_path = "inputs/dataset.csv"

In [3]:
df = pd.read_csv(dataset_path)
archaic_sentences = df["Sentence"].dropna().tolist()[:20]

# Fine Tuning

## Fine tuning parameters

In [ ]:
# Path to csvs folder
csvs_folder = "fine_tuning/csvs"  
# Path to "dataset_concatenato" folder
dataset_conc_folder = "inputs/dataset_concatenato.csv"

training_epochs=15

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

new_token_max=50
temperature=0.7
max_translations=0 # 0 = no limit
def getPrompt(archaic_sentence):
    prompt = (
        "You are a professional translator of ancient texts into modern Italian.\n"
        "Transform the following archaic sentence into modern Italian, preserving its meaning.\n"
        f"Archaic text: {archaic_sentence}\n"
        "Modern translation:"
    )

    return prompt

## Load Dataset

In [ ]:

# Load all csv files
csv_files = glob.glob(os.path.join(csvs_folder, "*.csv"))

# Load and concatenate
dataframes = []
for file in csv_files:
    try:
        df = pd.read_csv(file)
        dataframes.append(df)
    except Exception as e:
        print(f"Error in file {file}: {e}")

# Vertical Union
df_final = pd.concat(dataframes, ignore_index=True)

# Save
df_final.to_csv(dataset_conc_folder, index=False)

dataset = load_dataset("csv", data_files=dataset_conc_folder)["train"]
dataset = dataset.train_test_split(test_size=0.1)



## Model & Tokenizer

In [ ]:

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)


# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast=True)

tokenizer.pad_token = tokenizer.eos_token  #to avoid padding issues
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model.is_loaded_in_4bit = True


## Lora Config

In [ ]:
peft_config = LoraConfig(
    r=8, 
    lora_alpha=16,
    task_type=TaskType.CAUSAL_LM,
    lora_dropout=0.1,
    bias="none"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


## Preprocessing dataset

In [ ]:
def format_prompt(example):
    prompt = getPrompt(example['text'])
    return {
        "input_ids": tokenizer(prompt, truncation=True, padding="max_length", max_length=512)["input_ids"],
        "labels": tokenizer(example["translation"], truncation=True, padding="max_length", max_length=512)["input_ids"]
    }

tokenized_dataset = {
    "train": dataset["train"].map(format_prompt, remove_columns=dataset["train"].column_names),
    "test": dataset["test"].map(format_prompt, remove_columns=dataset["test"].column_names)
}

## Setup Trainer

In [ ]:
logging.set_verbosity_debug()
training_args = TrainingArguments(
    output_dir="Models/Mistral/mistral-lora-itmoderno",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=training_epochs,
    learning_rate=2e-4,
    fp16=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=20,
    disable_tqdm=False,    
    report_to="none",         
    logging_dir="./logs", 
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
def compute_metrics(eval_preds):
    return {} 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


## Start Fine Tuning & Save

In [ ]:
trainer.train()
model.save_pretrained("Models/Mistral/mistral-finetuned-itmodern")
tokenizer.save_pretrained("Models/Mistral/mistral-finetuned-itmodern")
from peft import PeftModel
from transformers import AutoModelForCausalLM

# Load base model
base_model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

# Load LoRa Checkpointq
lora_model = PeftModel.from_pretrained(base_model, "Models/Mistral/mistral-finetuned-itmodern")

# Merge LoRa weights into the base model
merged_model = lora_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("Models/Mistral/mistral-lora-merged")



# Translations

In [ ]:

# 0. Quantization config (bnb4bit)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

# 1. Select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Load Merged Model & Tokenizer
model_path = "Models/Mistral/mistral-lora-merged"
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")  # tokenizer originale
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto"
)
tokenizer.pad_token = tokenizer.eos_token
model.eval()

# 3. Load dataset
df = pd.read_csv("inputs/dataset.csv")
df["generated_translation"] = ""

# 4. Translation function
def translate(s):
    prompt = getPrompt(s)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)
    output = model.generate(
        **inputs,
        max_new_tokens=new_token_max,
        temperature=temperature,
        top_p=0.9,
        do_sample=True,
        use_cache=True
    )
    return tokenizer.decode(output[0], skip_special_tokens=True).split("Traduzione moderna:")[-1].strip()

# 5. Generation loop
results = []
for i, s in enumerate(tqdm(df["Sentence"].tolist())):
    if max_translations!= 0 and i >= max_translations:
        results.append("[SKIPPED]")
        continue
    try:
        results.append(translate(s))
    except:
        results.append("[ERROR]")

df["generated_translation"] = results
df["score_human"] = 0
# 6. Save results
df.to_csv("outputs/dataset_with_mistral_translations.csv", index=False)
